In [148]:
import os
import numpy as np
import matplotlib.pyplot as plt
import csv
import cv2 as cv
import pandas as pd
import shutil
import glob
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from PIL import Image

In [120]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=(150,150,3),activation='relu',))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=(150,150,3),activation='relu',))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=(150,150,3),activation='relu',))
model.add(MaxPooling2D(pool_size=(2,2)))

In [121]:
model.add(Flatten())

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(27, activation='softmax'))

In [122]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [123]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 17, 17, 32)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 9248)             

In [134]:
image_gen = ImageDataGenerator(rotation_range=30,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               rescale=1/255,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True,
                               fill_mode='nearest')

In [125]:
train_images = 'train_images/'
files = os.listdir(train_images)
data = 'data/'

In [126]:
df = pd.read_csv('train.csv')
df

,path,label
0,58a20c9a06df46da81135a567b670e27.jpg,Texaco
1,3095bf82cbef4ae2847742527821bec8.jpg,Pepsi
2,af2f28fb4de74945b4f6073b87c17c4e.jpg,Adidas
3,641c248fbdba45ba9868a27747ffcce9.jpg,Heineken
4,446d1c5e86324557b4a7c7aa12f2925e.jpg,Texaco
...,...,...
3995,f176ed47ba2a423fb7d95a7094a90bfb.jpg,Porsche
3996,a8ed79caf5f74173b4b1aac7bf694347.jpg,Unicef
3997,f7a6acbc89d542caae289ad325968193.jpg,Adidas
3998,b79af28fb37841efbef04bbff4f1315d.jpg,Vodafone


In [104]:
df['label'].value_counts()

Pepsi        353
Sprite       248
Cocacola     240
Vodafone     239
Adidas       222
Apple        221
Citroen      182
Unicef       173
Starbucks    172
Heineken     140
Nike         140
RedBull      125
Ford         113
Puma         112
McDonalds    111
Porsche      108
Intel        108
Texaco       107
DHL          105
BMW          105
Google       103
Yahoo        102
Fedex         99
Nbc           95
Ferrari       94
HP            93
Mini          90
Name: label, dtype: int64

In [92]:
df['label'] = df['label'].astype(str)

In [93]:
df['path'].values

array(['58a20c9a06df46da81135a567b670e27.jpg',
       '3095bf82cbef4ae2847742527821bec8.jpg',
       'af2f28fb4de74945b4f6073b87c17c4e.jpg', ...,
       'f7a6acbc89d542caae289ad325968193.jpg',
       'b79af28fb37841efbef04bbff4f1315d.jpg',
       '46c22d00c07b4677875ea2fa1308db96.jpg'], dtype=object)

In [94]:
num = 0
for file in df['path'].values:
    src_dir = f'{train_images}{file}'
    label = df[df['path'] == file]['label'][num]
    if num < 3200:
        dst_dir = f'{data}train/{label}/{file}'
    else:
        dst_dir = f'{data}test/{label}/{file}'
    os.makedirs(os.path.dirname(dst_dir), exist_ok=True)
    shutil.copy(src_dir, dst_dir)
    num += 1

In [138]:
image_gen_train = image_gen.flow_from_directory('data/train/',
                                                 target_size=(150, 150),
                                                 batch_size = 16,
                                                 class_mode = 'categorical',
                                                )


image_gen_test = image_gen.flow_from_directory('data/test/',
                                                 target_size=(150, 150),
                                                 batch_size = 16,
                                                 class_mode = 'categorical',
                                               )

Found 3200 images belonging to 27 classes.
Found 800 images belonging to 27 classes.


In [139]:
image_shape = (150,150,3)

In [140]:
image_gen

In [142]:
model.compile(loss='CategoricalCrossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [144]:
result = model.fit(image_gen_train, 
                   epochs = 30,
                   steps_per_epoch = 100,
                   validation_data = image_gen_test,
                   validation_steps = 12
                  )

Epoch 1/30
100/100 [==============================] - 34s 336ms/step - loss: 2.9179 - accuracy: 0.1975 - val_loss: 2.6602 - val_accuracy: 0.2708
Epoch 2/30
100/100 [==============================] - 35s 349ms/step - loss: 2.4669 - accuracy: 0.3106 - val_loss: 2.0100 - val_accuracy: 0.4531
Epoch 3/30
100/100 [==============================] - 36s 359ms/step - loss: 2.1540 - accuracy: 0.3750 - val_loss: 1.9641 - val_accuracy: 0.4219
Epoch 4/30
100/100 [==============================] - 37s 370ms/step - loss: 1.9187 - accuracy: 0.4356 - val_loss: 1.6515 - val_accuracy: 0.5312
Epoch 5/30
100/100 [==============================] - 39s 394ms/step - loss: 1.8701 - accuracy: 0.4762 - val_loss: 1.7616 - val_accuracy: 0.4375
Epoch 6/30
100/100 [==============================] - 39s 389ms/step - loss: 1.7442 - accuracy: 0.4875 - val_loss: 1.5590 - val_accuracy: 0.5573
Epoch 7/30
100/100 [==============================] - 41s 408ms/step - loss: 1.6061 - accuracy: 0.5312 - val_loss: 1.3436 - val_ac

In [145]:
result.history['accuracy']

[0.19750000536441803,
 0.31062498688697815,
 0.375,
 0.43562498688697815,
 0.4762499928474426,
 0.48750001192092896,
 0.53125,
 0.5537499785423279,
 0.5856249928474426,
 0.5849999785423279,
 0.6287500262260437,
 0.609375,
 0.637499988079071,
 0.6787499785423279,
 0.684374988079071,
 0.6937500238418579,
 0.7006250023841858,
 0.7106249928474426,
 0.6949999928474426,
 0.7318750023841858,
 0.7406250238418579,
 0.7406250238418579,
 0.7437499761581421,
 0.7543749809265137,
 0.7649999856948853,
 0.7599999904632568,
 0.7618749737739563,
 0.7699999809265137,
 0.7768750190734863,
 0.78125]

In [146]:
model.save('brand_recognition.h5')

In [149]:
model = keras.models.load_model('brand_recognition.h5')

In [150]:
image_gen_train.class_indices

{'Adidas': 0,
 'Apple': 1,
 'BMW': 2,
 'Citroen': 3,
 'Cocacola': 4,
 'DHL': 5,
 'Fedex': 6,
 'Ferrari': 7,
 'Ford': 8,
 'Google': 9,
 'HP': 10,
 'Heineken': 11,
 'Intel': 12,
 'McDonalds': 13,
 'Mini': 14,
 'Nbc': 15,
 'Nike': 16,
 'Pepsi': 17,
 'Porsche': 18,
 'Puma': 19,
 'RedBull': 20,
 'Sprite': 21,
 'Starbucks': 22,
 'Texaco': 23,
 'Unicef': 24,
 'Vodafone': 25,
 'Yahoo': 26}

In [162]:
brands = {
 0:'Adidas',
 1:'Apple',
 2:'BMW',
 3:'Citroen',
 4:'Cocacola',
 5:'DHL',
 6:'Fedex',
 7:'Ferrari',
 8:'Ford',
 9:'Google',
 10:'HP',
 11:'Heineken',
 12:'Intel',
 13:'McDonalds',
 14:'Mini',
 15:'Nbc',
 16:'Nike',
 17:'Pepsi',
 18:'Porsche',
 19:'Puma',
 20:'RedBull',
 21:'Sprite',
        
 22:'Starbucks',
 23:'Texaco',
 24:'Unicef',
 25:'Vodafone',
 26:'Yahoo'
}

In [166]:
dict_brands = []
for file in glob.glob("test_images/*.jpg"):
    img = image.load_img(file,target_size=(150,150))
    img = image.img_to_array(img)
    img = np.expand_dims(img,axis=0)
    img = img / 255

    pred_array = model.predict(img)
    print('It is', brands[np.argmax(pred_array)], 'with probability', pred_array.max())
    dict_brands.append({'file':file, 'brand':brands[np.argmax(pred_array)], 'prob':pred_array.max()})

It is Starbucks with probability 0.46938998
It is Yahoo with probability 0.9961349
It is Apple with probability 0.99629813
It is Cocacola with probability 0.99994826
It is Adidas with probability 0.99495715
It is Starbucks with probability 0.98419803
It is Puma with probability 0.2738276
It is Pepsi with probability 0.99999106
It is Heineken with probability 0.97384655
It is Ford with probability 0.4833323
It is BMW with probability 0.99764234
It is Ford with probability 0.9996338
It is Ferrari with probability 0.99958056
It is Porsche with probability 0.9997265
It is Puma with probability 0.9095948
It is Texaco with probability 0.9839844
It is Puma with probability 0.52214426
It is Fedex with probability 0.92320883
It is Vodafone with probability 0.9258314
It is Cocacola with probability 0.9637411
It is Pepsi with probability 1.0
It is Mini with probability 0.3942285
It is Citroen with probability 0.4200484
It is Starbucks with probability 0.46938998
It is Fedex with probability 0.999

In [155]:
type(training_set.class_indices)

dict

In [156]:
training_set.class_indices.keys()

dict_keys(['Adidas', 'Apple', 'BMW', 'Citroen', 'Cocacola', 'DHL', 'Fedex', 'Ferrari', 'Ford', 'Google', 'HP', 'Heineken', 'Intel', 'McDonalds', 'Mini', 'Nbc', 'Nike', 'Pepsi', 'Porsche', 'Puma', 'RedBull', 'Sprite', 'Starbucks', 'Texaco', 'Unicef', 'Vodafone', 'Yahoo'])

In [167]:
dict_brands

[{'file': 'test_images/2382afd2208b4f08a8758cbfa84b7ab2.jpg',
  'brand': 'Starbucks',
  'prob': 0.46938998},
 {'file': 'test_images/014baffbb4064ac9871a8c18643dedaf.jpg',
  'brand': 'Yahoo',
  'prob': 0.9961349},
 {'file': 'test_images/96e9dd37bccd4a9e908f31d331468e04.jpg',
  'brand': 'Apple',
  'prob': 0.99629813},
 {'file': 'test_images/a6541d71d57247e4a78ad72e31d24607.jpg',
  'brand': 'Cocacola',
  'prob': 0.99994826},
 {'file': 'test_images/022a8fba5d4e4c0d8ce5fc10eadd61b8.jpg',
  'brand': 'Adidas',
  'prob': 0.99495715},
 {'file': 'test_images/228ddef65d3147339669d3ff9a93f09a.jpg',
  'brand': 'Starbucks',
  'prob': 0.98419803},
 {'file': 'test_images/c13eafe7303444f093df8225a6034af5.jpg',
  'brand': 'Puma',
  'prob': 0.2738276},
 {'file': 'test_images/b31e3a985737413b9e0241476c247f57.jpg',
  'brand': 'Pepsi',
  'prob': 0.99999106},
 {'file': 'test_images/ad285d96b2b44ef9951d29340c7e193d.jpg',
  'brand': 'Heineken',
  'prob': 0.97384655},
 {'file': 'test_images/296f89362d3c422cb18b

In [168]:
df_subm = pd.DataFrame(dict_brands)

In [169]:
df_subm

,file,brand,prob
0,test_images/2382afd2208b4f08a8758cbfa84b7ab2.jpg,Starbucks,0.469390
1,test_images/014baffbb4064ac9871a8c18643dedaf.jpg,Yahoo,0.996135
2,test_images/96e9dd37bccd4a9e908f31d331468e04.jpg,Apple,0.996298
3,test_images/a6541d71d57247e4a78ad72e31d24607.jpg,Cocacola,0.999948
4,test_images/022a8fba5d4e4c0d8ce5fc10eadd61b8.jpg,Adidas,0.994957
...,...,...,...
527,test_images/187202217f694e8cbd9bbc3d6089564a.jpg,Cocacola,0.995053
528,test_images/d0d0a56b32dc4cd5ba2a8378674e9154.jpg,Cocacola,0.999982
529,test_images/36a9b6434e0e486683dcbfffc19c751b.jpg,Unicef,0.995021
530,test_images/2a3090ae36ee400496870583334438f5.jpg,Fedex,0.999997


In [171]:
del df_subm['prob']

In [172]:
df_subm.to_csv('results.csv')